In [7]:
from lxml import etree as ET
import pandas as pd
import os
import collections

In [8]:
dir_workspace = os.path.join(os.getcwd(),'workspace_tsd_transform')
file_jdoc = os.path.join(dir_workspace,'javadoc.xml')
file_tsd = os.path.join(dir_workspace,'VDDBForward.tsd')
file_out_tsd = os.path.join(dir_workspace,'VDDBForward_Out.tsd')
tree_jdoc = ET.parse(file_jdoc)
tree_tsd = ET.parse(file_tsd)

def get_method_name(elm_method):
    dataTypes = '['+ ','.join([elm_para.attrib['dataType'].lower() for elm_para in elm_method.findall('.//Parameter')])+']'
    return elm_method.attrib['methodName'] + '[' + str(len(elm_method.findall('.//Parameter'))) +']'+dataTypes

def get_method_impl_name(elm_step):
    dataTypes = '['+ ','.join([elm_para.attrib['format'].lower() for elm_para in elm_step.findall('.//UiParameter')])+']'
    return elm_step.attrib['stepName'] + '[' + str(len(elm_step.findall('.//UiParameter'))) +']'+dataTypes

dict_method_elm  = {get_method_name(elm):elm for elm in tree_jdoc.findall('.//Method')}

In [9]:
list_missed_method = []
for elm_step in tree_tsd.findall('.//UiTestStep'):
    method_impl_name = get_method_impl_name(elm_step)
    if(method_impl_name in dict_method_elm):
        list_method_params = dict_method_elm[method_impl_name].findall('.//Parameter')
        list_step_params = elm_step.findall('.//UiParameter')
        for i in range(0,len(list_step_params)):
            elm_m_param = list_method_params[i]
            elm_s_param = list_step_params[i]
            elm_s_param.attrib['name'] = elm_m_param.attrib['parameterName']
    else:
        list_missed_method.append(method_impl_name)
print(set(list_missed_method))

with open(file_out_tsd, "wb") as f:
    f.write(ET.tostring(tree_tsd, pretty_print = True))

{'RunSQL[3][string,string,string]', 'RunSQL[6][string,string,string,string,string,string]'}
